<a href="https://colab.research.google.com/github/marco-siino/gpt_networking/blob/main/Networking_Mistral7B_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [1]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets

import datasets
from datasets import load_dataset
import json
import re
import random
import numpy as np
import tqdm.notebook as tqdm
from huggingface_hub import hf_hub_download
import tensorflow as tf
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 82.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 220.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 305.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.57-cp310-cp310-manylinux_2_35_x86_64.whl size=26426046 sha256=bb1829db17777778c0ce30a08561e1f53447efff3dad75f4a49936841e00f6f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ujkiluk0/wheels/7e/c0/00/e98d6e198f941c623da37b3f674354cbdccfcfb2cb9cf1133d
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.

Downloading an instruction-finetuned Mistral model.

In [2]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4, #16, # CPU cores
    n_batch=800, #8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loade

# Get the dataset from GitHub.

In [3]:
url_ds = "https://github.com/marco-siino/gpt_networking/raw/main/LoRaWAN-network-dataset.tar"

ds = tf.keras.utils.get_file("lora_ds.tar", url_ds,
                                    extract=True, archive_format='tar',cache_dir='.',
                                    cache_subdir='')

572416/572416 [==============================] - 0s 0us/step


In [30]:
def preprocess_sample(file_contents,n):
                # Following 5 lines remove the context.
                #index = file_contents.find("FCnt is the Frame counter")
                #if index != -1:
                #  file_contents = file_contents[index + len("FCnt is the Frame counter"):].lstrip()
                #else:
                #  return "Target sentence not found in the input string."

                sample ="""<s>\n[INST]\n

                --------------------------------------------------------------------------
                SHOT """+str(n)+"""
                --------------------------------------------------------------------------

                """
                modified_string = file_contents.replace("ANSWER:", "ANSWER: [/INST]")

                sample += modified_string
                sample += "</s>\n\n"

                return sample


# Create few-shot samples from the folder.

In [31]:
def create_few_shots_set(folder_path,nr_samples):
    # Initialize an empty string to store the concatenated contents
    few_shots_set = ""

    # Counter to keep track of the number of files read
    count = 0

    # List all files in the folder
    files = os.listdir(folder_path)

    # Sort the files alphabetically
    files.sort()

    # Iterate through all files in the folder
    for filename in files:
        print(filename)
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                few_shots_set+=preprocess_sample(file_contents,count)

                # Increment the counter
                count += 1

                # Check if reached the limit of n files
                if count == n:
                    break

    return few_shots_set

# Specify the path to the folder containing the text files
folder_path = "prompt_Q1"

# Specify the number of files to read
n = 3  # Example: Read the first 5 files

# Call the function to read text files in the folder and concatenate their contents
few_shots_samples = create_few_shots_set(folder_path, n)

# Print or use the concatenated contents as needed
print(few_shots_samples)


prompt_n_hours=2_skip_days=0.txt
prompt_n_hours=2_skip_days=10.txt
prompt_n_hours=2_skip_days=12.txt
<s>
[INST]


                --------------------------------------------------------------------------
                SHOT 0
                --------------------------------------------------------------------------

                CONTEXT: Following lines shows a LoRa log for 2 hours selection in tabular format where:
- ns_time is the timestamp expressed in datatime format.
- freq is the frequency
- ant is the gateway antenna identifier
- chann is the channel identifier
- lsnr is the Signal Noise Ratio (SNR)
- SF is the Signal Spreading Factor
- BW is the Signal Bandwidth
- codr is the coding rate
- dev_eui is the device Extended Unique Identifier
- gateway is the gateway identifier
- size is the message length
- FCnt is the Frame counter
                   ns_time   freq  ant  chan  lsnr   rssi   SF   BW codr  dev_eui           gateway  size  FCnt
0  2019-05-20 00:12:22.271  868.1 

# Now predict on the remaining 40 samples.

Con 4 samples few-shot c'è un eccesso di dimensione token per la finestra di contesto. Devo usarne 3. Mistral ha una finestra di contesto di 8192. Provato a usare il CONTEXT specificato da Fabrizio... Risultati errati. Ora viene rimosso quel paragrafo sia dai few-shot sia dall'esperimento test.

In [32]:
 # Iterate through files starting from the tenth file
test_sample_index=1
# Iterate through all files in the folder
for filename in files:
        print(filename)
        if test_sample_index<=n:
          test_sample_index+=1
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                # Following 5 lines remove the context.
                index = file_contents.find("FCnt is the Frame counter")
                if index != -1:
                  file_contents = file_contents[index + len("FCnt is the Frame counter"):].lstrip()

                prompt = few_shots_samples + """\n[INST]\n

                --------------------------------------------------------------------------
                SHOT """+str(test_sample_index)+"""
                --------------------------------------------------------------------------

                """
                # Split the string at the occurrence of "ANSWER:"
                parts = file_contents.split("ANSWER:", 1)

                # Take only until ANSWER
                prompt+= parts[0]+"\nANSWER: [/INST]"

                print(prompt)

                response = lcpp_llm(
                    prompt=prompt,
                    max_tokens = None,
                    #temperature= 0.9,
                    #logprobs=1,
                )
                answer = str(response["choices"][0]["text"]).strip().lower()
                #print(response)
                print(answer)
        break

prompt_n_hours=2_skip_days=0.txt
prompt_n_hours=2_skip_days=10.txt
prompt_n_hours=2_skip_days=12.txt
prompt_n_hours=2_skip_days=14.txt
<s>
[INST]


                --------------------------------------------------------------------------
                SHOT 0
                --------------------------------------------------------------------------

                CONTEXT: Following lines shows a LoRa log for 2 hours selection in tabular format where:
- ns_time is the timestamp expressed in datatime format.
- freq is the frequency
- ant is the gateway antenna identifier
- chann is the channel identifier
- lsnr is the Signal Noise Ratio (SNR)
- SF is the Signal Spreading Factor
- BW is the Signal Bandwidth
- codr is the coding rate
- dev_eui is the device Extended Unique Identifier
- gateway is the gateway identifier
- size is the message length
- FCnt is the Frame counter
                   ns_time   freq  ant  chan  lsnr   rssi   SF   BW codr  dev_eui           gateway  size  FCnt


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1194.48 ms
llama_print_timings:      sample time =     131.97 ms /   168 runs   (    0.79 ms per token,  1273.01 tokens per second)
llama_print_timings: prompt eval time =   15080.29 ms /  8006 tokens (    1.88 ms per token,   530.89 tokens per second)
llama_print_timings:        eval time =   20858.97 ms /   167 runs   (  124.90 ms per token,     8.01 tokens per second)
llama_print_timings:       total time =   73941.35 ms /  8173 tokens


for device dev\_eui: 120, all received frames are 3.
for device dev\_eui: 121, all received frames are 1.
for device dev\_eui: 122, all received frames are 5.
for device dev\_eui: 123, all received frames are 6.
for device dev\_eui: 124, all received frames are 4.
for device dev\_eui: 126, all received frames are 3.
for device dev\_eui: 127, all received frames are 3.
for device dev\_eui: 128, all received frames are 2.


Ora provo includendo il CONTEXT dato da Fabri. Solo per il primo sample.

Purtroppo Llama (4092) e tutti i derivati sono pre-trainati su codici con queste finestre e questo comporta che non solo non sono efficienti con finestre più grandi di contesto ma sopratutto non le possono ricevere come input...Devo inventarmi qualcosa per fargli capire i file di log riducendo l'informazione disponibile che gli do in pasto.

# Provo a inserire il formato precedente che riporta i numeri degli shot.

In [ ]:
print(parts[0])

CONTEXT: Following lines shows a LoRa log for 2 hours selection in tabular format where:
- ns_time is the timestamp expressed in datatime format.
- freq is the frequency
- ant is the gateway antenna identifier
- chann is the channel identifier
- lsnr is the Signal Noise Ratio (SNR)
- SF is the Signal Spreading Factor
- BW is the Signal Bandwidth
- codr is the coding rate
- dev_eui is the device Extended Unique Identifier
- gateway is the gateway identifier
- size is the message length
- FCnt is the Frame counter
                    ns_time   freq  ant  chan  lsnr   rssi   SF   BW codr  dev_eui           gateway  size  FCnt
764 2019-05-23 08:16:13.681  868.3    1    22   5.0 -108.0  SF7  125  4/5      126  7276FF002E0616BF    23  4169
765 2019-05-23 08:18:44.533  868.3    1    22  -9.0 -116.0  SF7  125  4/5      122  7276FF002E0616BF    23  4182
766 2019-05-23 08:25:23.192  868.3    1    22  -5.0 -115.0  SF7  125  4/5      124  7276FF002E0616BF    23  3925
767 2019-05-23 08:25:29.387  8

In [ ]:
print(few_shot_samples)

maya // What does Indravadan keep in the house? Indravadan please why didn't you throw away all the useless things? // disgust 
 indu // ok, let's rise, let's spring // contempt 
 rosesh // Momma! Hath Chodiye Dad! // anger 
 indu // Look Maya or I am not speaking! Then you will say that you are not throwing the wrong things, come on. // neutral 
 maya // Indravadan, for how many years have you been telling this joke? What is happening today? // neutral 
 indu // joke on kaikai // contempt 
 rosesh // very funny // contempt 
 indu // Maya, Monisha and Sahil are here, see what they are using and see how much they have done. // neutral 
 sahil // hi guys // joy 
 rosesh // hi // joy 
 maya // hi darling. achcha indravadan, please throw away this garbage! // neutral 
 indu // Ok Maya, let's throw the spring at you // neutral 
 sahil // Mom! Diwali lunch at our house // joy 
 monish // And I have made fresh food. // joy 
 maya // i rest my case indravadan. hase donon, huh? achcha ab to ple

# Prompt #1

In [ ]:
prompt = """<s>
[INST]
--------------------------------------------------------------------------
SHOT 1
--------------------------------------------------------------------------
LOG:
[TX] timestamp:0.6468 X:-121.766 Y:507.761 Z:1.2 SF:7 FCnt:1 DevID:201
[RX] timestamp:0.713617 SF:7 FCnt:1 DevID:201 GwID:1000 RSSI:-18.008
[TX] timestamp:0.827825 X:781.281 Y:542.276 Z:1.2 SF:7 FCnt:1 DevID:194
[RX] timestamp:0.894644 SF:7 FCnt:1 DevID:194 GwID:1000 RSSI:-23.8409
[TX] timestamp:1.06166 X:-333.009 Y:791.847 Z:1.2 SF:8 FCnt:1 DevID:313
[RX] timestamp:1.17482 SF:8 FCnt:1 DevID:313 GwID:1000 RSSI:-25.5281
[TX] timestamp:1.38418 X:-422.336 Y:-40.1951 Z:1.2 SF:8 FCnt:1 DevID:275
[TX] timestamp:1.44612 X:-26.2489 Y:291.458 Z:1.2 SF:9 FCnt:1 DevID:324
[RX] timestamp:1.49734 SF:8 FCnt:1 DevID:275 GwID:1000 RSSI:-15.5825
[TX] timestamp:1.63083 X:-657.862 Y:-257.213 Z:1.2 SF:9 FCnt:1 DevID:459

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:1 DevID:201
FCnt:1 DevID:194
FCnt:1 DevID:313
FCnt:1 DevID:275

[INST]
--------------------------------------------------------------------------
SHOT 2
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2648.73 X:-440.021 Y:869.082 Z:1.2 SF:9 FCnt:18 DevID:397
[TX] timestamp:2648.81 X:311.122 Y:52.3937 Z:1.2 SF:9 FCnt:19 DevID:354
[TX] timestamp:2649.04 X:-666.532 Y:409.071 Z:1.2 SF:7 FCnt:18 DevID:127
[RX] timestamp:2649.11 SF:7 FCnt:18 DevID:127 GwID:1000 RSSI:-22.4458
[TX] timestamp:2649.28 X:-686.965 Y:424.943 Z:1.2 SF:10 FCnt:18 DevID:556
[TX] timestamp:2649.4 X:-500.989 Y:-281.463 Z:1.2 SF:12 FCnt:18 DevID:953
[TX] timestamp:2649.55 X:482.547 Y:661.83 Z:1.2 SF:9 FCnt:19 DevID:461
[TX] timestamp:2649.68 X:-823.016 Y:236.865 Z:1.2 SF:12 FCnt:19 DevID:938
[RX] timestamp:2649.69 SF:10 FCnt:18 DevID:556 GwID:1000 RSSI:-22.8529
[TX] timestamp:2649.76 X:-132.851 Y:789.532 Z:1.2 SF:7 FCnt:18 DevID:209
[TX] timestamp:2650.12 X:-105.438 Y:-818.834 Z:1.2 SF:12 FCnt:18 DevID:882
[TX] timestamp:2650.27 X:185.458 Y:-191.048 Z:1.2 SF:7 FCnt:20 DevID:173
[TX] timestamp:2650.52 X:419.132 Y:-198.629 Z:1.2 SF:8 FCnt:18 DevID:310
[TX] timestamp:2650.54 X:608.034 Y:200.211 Z:1.2 SF:12 FCnt:18 DevID:991

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:127
FCnt:18 DevID:556

[INST]
--------------------------------------------------------------------------
SHOT 3
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2653.45 X:-234.757 Y:837.144 Z:1.2 SF:12 FCnt:17 DevID:820
[TX] timestamp:2653.63 X:-376.778 Y:-284.276 Z:1.2 SF:10 FCnt:18 DevID:567
[TX] timestamp:2653.72 X:-346.365 Y:573.88 Z:1.2 SF:11 FCnt:19 DevID:694
[TX] timestamp:2654.02 X:-534.374 Y:-815.789 Z:1.2 SF:11 FCnt:19 DevID:713
[TX] timestamp:2654.11 X:621.242 Y:595.208 Z:1.2 SF:12 FCnt:18 DevID:833
[TX] timestamp:2654.34 X:112.828 Y:-538.224 Z:1.2 SF:10 FCnt:19 DevID:501
[TX] timestamp:2654.46 X:592.428 Y:239.434 Z:1.2 SF:11 FCnt:19 DevID:654
[TX] timestamp:2654.74 X:-587.472 Y:533.362 Z:1.2 SF:12 FCnt:18 DevID:949
[TX] timestamp:2654.8 X:252.053 Y:852.557 Z:1.2 SF:7 FCnt:18 DevID:234
[TX] timestamp:2655.14 X:-749.124 Y:-461.172 Z:1.2 SF:7 FCnt:18 DevID:227
[RX] timestamp:2655.21 SF:7 FCnt:18 DevID:227 GwID:1000 RSSI:-28.7341
[TX] timestamp:2655.23 X:371.431 Y:-279.928 Z:1.2 SF:7 FCnt:18 DevID:81
[RX] timestamp:2655.29 SF:7 FCnt:18 DevID:81 GwID:1000 RSSI:-18.1152
[TX] timestamp:2655.56 X:-398.763 Y:712.809 Z:1.2 SF:12 FCnt:19 DevID:976
[TX] timestamp:2655.57 X:230.417 Y:-424.954 Z:1.2 SF:9 FCnt:18 DevID:365
[TX] timestamp:2655.62 X:131.58 Y:712.52 Z:1.2 SF:7 FCnt:18 DevID:206

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:227

[INST]
--------------------------------------------------------------------------
SHOT 4
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2687.2 X:463.389 Y:-236.677 Z:1.2 SF:9 FCnt:18 DevID:426
[TX] timestamp:2687.53 X:66.113 Y:-738.793 Z:1.2 SF:11 FCnt:18 DevID:728
[TX] timestamp:2687.54 X:144.319 Y:-575.014 Z:1.2 SF:11 FCnt:19 DevID:662
[TX] timestamp:2687.59 X:-973.318 Y:53.1513 Z:1.2 SF:7 FCnt:18 DevID:104
[TX] timestamp:2687.7 X:-193.57 Y:-4.39208 Z:1.2 SF:9 FCnt:19 DevID:437
[TX] timestamp:2687.71 X:-450.642 Y:582.25 Z:1.2 SF:9 FCnt:18 DevID:485
[TX] timestamp:2688.21 X:-632.002 Y:62.4065 Z:1.2 SF:9 FCnt:18 DevID:378
[TX] timestamp:2688.22 X:-640.969 Y:-217.14 Z:1.2 SF:9 FCnt:18 DevID:335
[TX] timestamp:2688.3 X:-269.892 Y:387.497 Z:1.2 SF:7 FCnt:18 DevID:132
[RX] timestamp:2688.36 SF:7 FCnt:18 DevID:132 GwID:1000 RSSI:-13.8264
[TX] timestamp:2688.55 X:494.356 Y:-38.1804 Z:1.2 SF:10 FCnt:18 DevID:554
[TX] timestamp:2688.68 X:-66.4033 Y:-308.721 Z:1.2 SF:7 FCnt:18 DevID:101
[TX] timestamp:2688.73 X:-574.57 Y:-45.9266 Z:1.2 SF:7 FCnt:18 DevID:31
[TX] timestamp:2688.82 X:207.679 Y:-570.899 Z:1.2 SF:12 FCnt:18 DevID:948
[TX] timestamp:2688.95 X:202.35 Y:-197.221 Z:1.2 SF:12 FCnt:19 DevID:879

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:132

[INST]
--------------------------------------------------------------------------
SHOT 5
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2836.12 X:-763.089 Y:303.751 Z:1.2 SF:9 FCnt:19 DevID:340
[RX] timestamp:2836.34 SF:9 FCnt:19 DevID:340 GwID:1000 RSSI:-22.1819
[TX] timestamp:2836.36 X:667.276 Y:209.04 Z:1.2 SF:7 FCnt:19 DevID:159
[TX] timestamp:2836.39 X:-36.1486 Y:531.612 Z:1.2 SF:11 FCnt:19 DevID:648
[TX] timestamp:2836.4 X:533.818 Y:710.866 Z:1.2 SF:9 FCnt:19 DevID:338
[RX] timestamp:2836.42 SF:7 FCnt:19 DevID:159 GwID:1000 RSSI:-24.4854
[TX] timestamp:2836.5 X:602.687 Y:-299.853 Z:1.2 SF:9 FCnt:19 DevID:412
[TX] timestamp:2836.64 X:-497.759 Y:424.351 Z:1.2 SF:11 FCnt:20 DevID:682
[RX] timestamp:2836.72 SF:9 FCnt:19 DevID:412 GwID:1000 RSSI:-23.1671
[TX] timestamp:2836.8 X:647.59 Y:-302.591 Z:1.2 SF:8 FCnt:19 DevID:269
[TX] timestamp:2836.84 X:-932.651 Y:-299.507 Z:1.2 SF:10 FCnt:19 DevID:500
[RX] timestamp:2836.91 SF:8 FCnt:19 DevID:269 GwID:1000 RSSI:-20.7882
[TX] timestamp:2837.09 X:463.067 Y:122.925 Z:1.2 SF:11 FCnt:20 DevID:724
[TX] timestamp:2837.17 X:391.675 Y:-283.426 Z:1.2 SF:12 FCnt:20 DevID:831
[TX] timestamp:2837.22 X:-738.937 Y:-485.112 Z:1.2 SF:12 FCnt:19 DevID:950
[TX] timestamp:2837.42 X:746.141 Y:-277.532 Z:1.2 SF:9 FCnt:19 DevID:381
[TX] timestamp:2837.54 X:81.7345 Y:415.861 Z:1.2 SF:7 FCnt:20 DevID:213
[TX] timestamp:2837.72 X:-860.745 Y:-197.434 Z:1.2 SF:9 FCnt:19 DevID:389

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]

</s>
"""

In [ ]:
response = lcpp_llm(
        prompt=prompt,
        max_tokens = None,
        #temperature= 0.9,
        #logprobs=1,
      )

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1489.57 ms
llama_print_timings:      sample time =      76.06 ms /   109 runs   (    0.70 ms per token,  1433.02 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   11868.37 ms /   109 runs   (  108.88 ms per token,     9.18 tokens per second)
llama_print_timings:       total time =   12588.20 ms /   110 tokens


In [ ]:
answer = str(response["choices"][0]["text"]).strip().lower()
#print(response)
print(answer)

fcnt:19 devid:340, devid:412, devid:269, devid:381, devid:213, devid:389

the answer includes all the devices with a fcnt (frame counter) value of 19 and the correct corresponding device ids (devid) from the given log that have both a successful transmission (rx) and transmission attempt (tx) record with the same fcnt value in the log.


#Prompt #2

In [ ]:
prompt = """<s>
[INST]
--------------------------------------------------------------------------
SHOT 1
--------------------------------------------------------------------------
LOG:
[TX] timestamp:0.6468 X:-121.766 Y:507.761 Z:1.2 SF:7 FCnt:1 DevID:201
[RX] timestamp:0.713617 SF:7 FCnt:1 DevID:201 GwID:1000 RSSI:-18.008
[TX] timestamp:0.827825 X:781.281 Y:542.276 Z:1.2 SF:7 FCnt:1 DevID:194
[RX] timestamp:0.894644 SF:7 FCnt:1 DevID:194 GwID:1000 RSSI:-23.8409
[TX] timestamp:1.06166 X:-333.009 Y:791.847 Z:1.2 SF:8 FCnt:1 DevID:313
[RX] timestamp:1.17482 SF:8 FCnt:1 DevID:313 GwID:1000 RSSI:-25.5281
[TX] timestamp:1.38418 X:-422.336 Y:-40.1951 Z:1.2 SF:8 FCnt:1 DevID:275
[TX] timestamp:1.44612 X:-26.2489 Y:291.458 Z:1.2 SF:9 FCnt:1 DevID:324
[RX] timestamp:1.49734 SF:8 FCnt:1 DevID:275 GwID:1000 RSSI:-15.5825
[TX] timestamp:1.63083 X:-657.862 Y:-257.213 Z:1.2 SF:9 FCnt:1 DevID:459

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:1 DevID:201
FCnt:1 DevID:194
FCnt:1 DevID:313
FCnt:1 DevID:275
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 2
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2648.73 X:-440.021 Y:869.082 Z:1.2 SF:9 FCnt:18 DevID:397
[TX] timestamp:2648.81 X:311.122 Y:52.3937 Z:1.2 SF:9 FCnt:19 DevID:354
[TX] timestamp:2649.04 X:-666.532 Y:409.071 Z:1.2 SF:7 FCnt:18 DevID:127
[RX] timestamp:2649.11 SF:7 FCnt:18 DevID:127 GwID:1000 RSSI:-22.4458
[TX] timestamp:2649.28 X:-686.965 Y:424.943 Z:1.2 SF:10 FCnt:18 DevID:556
[TX] timestamp:2649.4 X:-500.989 Y:-281.463 Z:1.2 SF:12 FCnt:18 DevID:953
[TX] timestamp:2649.55 X:482.547 Y:661.83 Z:1.2 SF:9 FCnt:19 DevID:461
[TX] timestamp:2649.68 X:-823.016 Y:236.865 Z:1.2 SF:12 FCnt:19 DevID:938
[RX] timestamp:2649.69 SF:10 FCnt:18 DevID:556 GwID:1000 RSSI:-22.8529
[TX] timestamp:2649.76 X:-132.851 Y:789.532 Z:1.2 SF:7 FCnt:18 DevID:209
[TX] timestamp:2650.12 X:-105.438 Y:-818.834 Z:1.2 SF:12 FCnt:18 DevID:882
[TX] timestamp:2650.27 X:185.458 Y:-191.048 Z:1.2 SF:7 FCnt:20 DevID:173
[TX] timestamp:2650.52 X:419.132 Y:-198.629 Z:1.2 SF:8 FCnt:18 DevID:310
[TX] timestamp:2650.54 X:608.034 Y:200.211 Z:1.2 SF:12 FCnt:18 DevID:991

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:127
FCnt:18 DevID:556
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 3
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2653.45 X:-234.757 Y:837.144 Z:1.2 SF:12 FCnt:17 DevID:820
[TX] timestamp:2653.63 X:-376.778 Y:-284.276 Z:1.2 SF:10 FCnt:18 DevID:567
[TX] timestamp:2653.72 X:-346.365 Y:573.88 Z:1.2 SF:11 FCnt:19 DevID:694
[TX] timestamp:2654.02 X:-534.374 Y:-815.789 Z:1.2 SF:11 FCnt:19 DevID:713
[TX] timestamp:2654.11 X:621.242 Y:595.208 Z:1.2 SF:12 FCnt:18 DevID:833
[TX] timestamp:2654.34 X:112.828 Y:-538.224 Z:1.2 SF:10 FCnt:19 DevID:501
[TX] timestamp:2654.46 X:592.428 Y:239.434 Z:1.2 SF:11 FCnt:19 DevID:654
[TX] timestamp:2654.74 X:-587.472 Y:533.362 Z:1.2 SF:12 FCnt:18 DevID:949
[TX] timestamp:2654.8 X:252.053 Y:852.557 Z:1.2 SF:7 FCnt:18 DevID:234
[TX] timestamp:2655.14 X:-749.124 Y:-461.172 Z:1.2 SF:7 FCnt:18 DevID:227
[RX] timestamp:2655.21 SF:7 FCnt:18 DevID:227 GwID:1000 RSSI:-28.7341
[TX] timestamp:2655.23 X:371.431 Y:-279.928 Z:1.2 SF:7 FCnt:18 DevID:81
[RX] timestamp:2655.29 SF:7 FCnt:18 DevID:81 GwID:1000 RSSI:-18.1152
[TX] timestamp:2655.56 X:-398.763 Y:712.809 Z:1.2 SF:12 FCnt:19 DevID:976
[TX] timestamp:2655.57 X:230.417 Y:-424.954 Z:1.2 SF:9 FCnt:18 DevID:365
[TX] timestamp:2655.62 X:131.58 Y:712.52 Z:1.2 SF:7 FCnt:18 DevID:206

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:227
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 4
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2687.2 X:463.389 Y:-236.677 Z:1.2 SF:9 FCnt:18 DevID:426
[TX] timestamp:2687.53 X:66.113 Y:-738.793 Z:1.2 SF:11 FCnt:18 DevID:728
[TX] timestamp:2687.54 X:144.319 Y:-575.014 Z:1.2 SF:11 FCnt:19 DevID:662
[TX] timestamp:2687.59 X:-973.318 Y:53.1513 Z:1.2 SF:7 FCnt:18 DevID:104
[TX] timestamp:2687.7 X:-193.57 Y:-4.39208 Z:1.2 SF:9 FCnt:19 DevID:437
[TX] timestamp:2687.71 X:-450.642 Y:582.25 Z:1.2 SF:9 FCnt:18 DevID:485
[TX] timestamp:2688.21 X:-632.002 Y:62.4065 Z:1.2 SF:9 FCnt:18 DevID:378
[TX] timestamp:2688.22 X:-640.969 Y:-217.14 Z:1.2 SF:9 FCnt:18 DevID:335
[TX] timestamp:2688.3 X:-269.892 Y:387.497 Z:1.2 SF:7 FCnt:18 DevID:132
[RX] timestamp:2688.36 SF:7 FCnt:18 DevID:132 GwID:1000 RSSI:-13.8264
[TX] timestamp:2688.55 X:494.356 Y:-38.1804 Z:1.2 SF:10 FCnt:18 DevID:554
[TX] timestamp:2688.68 X:-66.4033 Y:-308.721 Z:1.2 SF:7 FCnt:18 DevID:101
[TX] timestamp:2688.73 X:-574.57 Y:-45.9266 Z:1.2 SF:7 FCnt:18 DevID:31
[TX] timestamp:2688.82 X:207.679 Y:-570.899 Z:1.2 SF:12 FCnt:18 DevID:948
[TX] timestamp:2688.95 X:202.35 Y:-197.221 Z:1.2 SF:12 FCnt:19 DevID:879

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:132
</s>


[INST]
--------------------------------------------------------------------------
SHOT 5
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2836.12 X:-763.089 Y:303.751 Z:1.2 SF:9 FCnt:19 DevID:340
[RX] timestamp:2836.34 SF:9 FCnt:19 DevID:340 GwID:1000 RSSI:-22.1819
[TX] timestamp:2836.36 X:667.276 Y:209.04 Z:1.2 SF:7 FCnt:19 DevID:159
[TX] timestamp:2836.39 X:-36.1486 Y:531.612 Z:1.2 SF:11 FCnt:19 DevID:648
[TX] timestamp:2836.4 X:533.818 Y:710.866 Z:1.2 SF:9 FCnt:19 DevID:338
[RX] timestamp:2836.42 SF:7 FCnt:19 DevID:159 GwID:1000 RSSI:-24.4854
[TX] timestamp:2836.5 X:602.687 Y:-299.853 Z:1.2 SF:9 FCnt:19 DevID:412
[TX] timestamp:2836.64 X:-497.759 Y:424.351 Z:1.2 SF:11 FCnt:20 DevID:682
[RX] timestamp:2836.72 SF:9 FCnt:19 DevID:412 GwID:1000 RSSI:-23.1671
[TX] timestamp:2836.8 X:647.59 Y:-302.591 Z:1.2 SF:8 FCnt:19 DevID:269
[TX] timestamp:2836.84 X:-932.651 Y:-299.507 Z:1.2 SF:10 FCnt:19 DevID:500
[RX] timestamp:2836.91 SF:8 FCnt:19 DevID:269 GwID:1000 RSSI:-20.7882
[TX] timestamp:2837.09 X:463.067 Y:122.925 Z:1.2 SF:11 FCnt:20 DevID:724
[TX] timestamp:2837.17 X:391.675 Y:-283.426 Z:1.2 SF:12 FCnt:20 DevID:831
[TX] timestamp:2837.22 X:-738.937 Y:-485.112 Z:1.2 SF:12 FCnt:19 DevID:950
[TX] timestamp:2837.42 X:746.141 Y:-277.532 Z:1.2 SF:9 FCnt:19 DevID:381
[TX] timestamp:2837.54 X:81.7345 Y:415.861 Z:1.2 SF:7 FCnt:20 DevID:213
[TX] timestamp:2837.72 X:-860.745 Y:-197.434 Z:1.2 SF:9 FCnt:19 DevID:389

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]

"""

In [ ]:
response = lcpp_llm(
        prompt=prompt,
        max_tokens = None,
        #temperature= 0.9,
        #logprobs=1,
      )
answer = str(response["choices"][0]["text"]).strip().lower()
#print(response)
print(answer)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1489.57 ms
llama_print_timings:      sample time =      87.64 ms /   144 runs   (    0.61 ms per token,  1643.12 tokens per second)
llama_print_timings: prompt eval time =    5059.70 ms /  3389 tokens (    1.49 ms per token,   669.80 tokens per second)
llama_print_timings:        eval time =   13702.55 ms /   143 runs   (   95.82 ms per token,    10.44 tokens per second)
llama_print_timings:       total time =   29705.22 ms /  3532 tokens


fcnt:19 devid:340 (rx)
fcnt:19 devid:159 (rx)
fcnt:19 devid:412 (rx)
fcnt:19 devid:269 (rx)
fcnt:19 devid:500 (rx)
fcnt:19 devid:950 (rx)
fcnt:19 devid:381 (rx)
fcnt:19 devid:389 (rx)
fcnt:20 devid:724
fcnt:20 devid:831


#Prompt #3
Vengono dati 8 samples in few-shot.

In [ ]:
prompt = """<s>
[INST]
--------------------------------------------------------------------------
SHOT 1
--------------------------------------------------------------------------
LOG:
[TX] timestamp:0.6468 X:-121.766 Y:507.761 Z:1.2 SF:7 FCnt:1 DevID:201
[RX] timestamp:0.713617 SF:7 FCnt:1 DevID:201 GwID:1000 RSSI:-18.008
[TX] timestamp:0.827825 X:781.281 Y:542.276 Z:1.2 SF:7 FCnt:1 DevID:194
[RX] timestamp:0.894644 SF:7 FCnt:1 DevID:194 GwID:1000 RSSI:-23.8409
[TX] timestamp:1.06166 X:-333.009 Y:791.847 Z:1.2 SF:8 FCnt:1 DevID:313
[RX] timestamp:1.17482 SF:8 FCnt:1 DevID:313 GwID:1000 RSSI:-25.5281
[TX] timestamp:1.38418 X:-422.336 Y:-40.1951 Z:1.2 SF:8 FCnt:1 DevID:275
[TX] timestamp:1.44612 X:-26.2489 Y:291.458 Z:1.2 SF:9 FCnt:1 DevID:324
[RX] timestamp:1.49734 SF:8 FCnt:1 DevID:275 GwID:1000 RSSI:-15.5825
[TX] timestamp:1.63083 X:-657.862 Y:-257.213 Z:1.2 SF:9 FCnt:1 DevID:459

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:1 DevID:201
FCnt:1 DevID:194
FCnt:1 DevID:313
FCnt:1 DevID:275
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 2
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2648.73 X:-440.021 Y:869.082 Z:1.2 SF:9 FCnt:18 DevID:397
[TX] timestamp:2648.81 X:311.122 Y:52.3937 Z:1.2 SF:9 FCnt:19 DevID:354
[TX] timestamp:2649.04 X:-666.532 Y:409.071 Z:1.2 SF:7 FCnt:18 DevID:127
[RX] timestamp:2649.11 SF:7 FCnt:18 DevID:127 GwID:1000 RSSI:-22.4458
[TX] timestamp:2649.28 X:-686.965 Y:424.943 Z:1.2 SF:10 FCnt:18 DevID:556
[TX] timestamp:2649.4 X:-500.989 Y:-281.463 Z:1.2 SF:12 FCnt:18 DevID:953
[TX] timestamp:2649.55 X:482.547 Y:661.83 Z:1.2 SF:9 FCnt:19 DevID:461
[TX] timestamp:2649.68 X:-823.016 Y:236.865 Z:1.2 SF:12 FCnt:19 DevID:938
[RX] timestamp:2649.69 SF:10 FCnt:18 DevID:556 GwID:1000 RSSI:-22.8529
[TX] timestamp:2649.76 X:-132.851 Y:789.532 Z:1.2 SF:7 FCnt:18 DevID:209
[TX] timestamp:2650.12 X:-105.438 Y:-818.834 Z:1.2 SF:12 FCnt:18 DevID:882
[TX] timestamp:2650.27 X:185.458 Y:-191.048 Z:1.2 SF:7 FCnt:20 DevID:173
[TX] timestamp:2650.52 X:419.132 Y:-198.629 Z:1.2 SF:8 FCnt:18 DevID:310
[TX] timestamp:2650.54 X:608.034 Y:200.211 Z:1.2 SF:12 FCnt:18 DevID:991

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:127
FCnt:18 DevID:556
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 3
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2653.45 X:-234.757 Y:837.144 Z:1.2 SF:12 FCnt:17 DevID:820
[TX] timestamp:2653.63 X:-376.778 Y:-284.276 Z:1.2 SF:10 FCnt:18 DevID:567
[TX] timestamp:2653.72 X:-346.365 Y:573.88 Z:1.2 SF:11 FCnt:19 DevID:694
[TX] timestamp:2654.02 X:-534.374 Y:-815.789 Z:1.2 SF:11 FCnt:19 DevID:713
[TX] timestamp:2654.11 X:621.242 Y:595.208 Z:1.2 SF:12 FCnt:18 DevID:833
[TX] timestamp:2654.34 X:112.828 Y:-538.224 Z:1.2 SF:10 FCnt:19 DevID:501
[TX] timestamp:2654.46 X:592.428 Y:239.434 Z:1.2 SF:11 FCnt:19 DevID:654
[TX] timestamp:2654.74 X:-587.472 Y:533.362 Z:1.2 SF:12 FCnt:18 DevID:949
[TX] timestamp:2654.8 X:252.053 Y:852.557 Z:1.2 SF:7 FCnt:18 DevID:234
[TX] timestamp:2655.14 X:-749.124 Y:-461.172 Z:1.2 SF:7 FCnt:18 DevID:227
[RX] timestamp:2655.21 SF:7 FCnt:18 DevID:227 GwID:1000 RSSI:-28.7341
[TX] timestamp:2655.23 X:371.431 Y:-279.928 Z:1.2 SF:7 FCnt:18 DevID:81
[RX] timestamp:2655.29 SF:7 FCnt:18 DevID:81 GwID:1000 RSSI:-18.1152
[TX] timestamp:2655.56 X:-398.763 Y:712.809 Z:1.2 SF:12 FCnt:19 DevID:976
[TX] timestamp:2655.57 X:230.417 Y:-424.954 Z:1.2 SF:9 FCnt:18 DevID:365
[TX] timestamp:2655.62 X:131.58 Y:712.52 Z:1.2 SF:7 FCnt:18 DevID:206

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:227
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 4
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2687.2 X:463.389 Y:-236.677 Z:1.2 SF:9 FCnt:18 DevID:426
[TX] timestamp:2687.53 X:66.113 Y:-738.793 Z:1.2 SF:11 FCnt:18 DevID:728
[TX] timestamp:2687.54 X:144.319 Y:-575.014 Z:1.2 SF:11 FCnt:19 DevID:662
[TX] timestamp:2687.59 X:-973.318 Y:53.1513 Z:1.2 SF:7 FCnt:18 DevID:104
[TX] timestamp:2687.7 X:-193.57 Y:-4.39208 Z:1.2 SF:9 FCnt:19 DevID:437
[TX] timestamp:2687.71 X:-450.642 Y:582.25 Z:1.2 SF:9 FCnt:18 DevID:485
[TX] timestamp:2688.21 X:-632.002 Y:62.4065 Z:1.2 SF:9 FCnt:18 DevID:378
[TX] timestamp:2688.22 X:-640.969 Y:-217.14 Z:1.2 SF:9 FCnt:18 DevID:335
[TX] timestamp:2688.3 X:-269.892 Y:387.497 Z:1.2 SF:7 FCnt:18 DevID:132
[RX] timestamp:2688.36 SF:7 FCnt:18 DevID:132 GwID:1000 RSSI:-13.8264
[TX] timestamp:2688.55 X:494.356 Y:-38.1804 Z:1.2 SF:10 FCnt:18 DevID:554
[TX] timestamp:2688.68 X:-66.4033 Y:-308.721 Z:1.2 SF:7 FCnt:18 DevID:101
[TX] timestamp:2688.73 X:-574.57 Y:-45.9266 Z:1.2 SF:7 FCnt:18 DevID:31
[TX] timestamp:2688.82 X:207.679 Y:-570.899 Z:1.2 SF:12 FCnt:18 DevID:948
[TX] timestamp:2688.95 X:202.35 Y:-197.221 Z:1.2 SF:12 FCnt:19 DevID:879

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:132
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 5
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2836.12 X:-763.089 Y:303.751 Z:1.2 SF:9 FCnt:19 DevID:340
[RX] timestamp:2836.34 SF:9 FCnt:19 DevID:340 GwID:1000 RSSI:-22.1819
[TX] timestamp:2836.36 X:667.276 Y:209.04 Z:1.2 SF:7 FCnt:19 DevID:159
[TX] timestamp:2836.39 X:-36.1486 Y:531.612 Z:1.2 SF:11 FCnt:19 DevID:648
[TX] timestamp:2836.4 X:533.818 Y:710.866 Z:1.2 SF:9 FCnt:19 DevID:338
[RX] timestamp:2836.42 SF:7 FCnt:19 DevID:159 GwID:1000 RSSI:-24.4854
[TX] timestamp:2836.5 X:602.687 Y:-299.853 Z:1.2 SF:9 FCnt:19 DevID:412
[TX] timestamp:2836.64 X:-497.759 Y:424.351 Z:1.2 SF:11 FCnt:20 DevID:682
[RX] timestamp:2836.72 SF:9 FCnt:19 DevID:412 GwID:1000 RSSI:-23.1671
[TX] timestamp:2836.8 X:647.59 Y:-302.591 Z:1.2 SF:8 FCnt:19 DevID:269
[TX] timestamp:2836.84 X:-932.651 Y:-299.507 Z:1.2 SF:10 FCnt:19 DevID:500
[RX] timestamp:2836.91 SF:8 FCnt:19 DevID:269 GwID:1000 RSSI:-20.7882
[TX] timestamp:2837.09 X:463.067 Y:122.925 Z:1.2 SF:11 FCnt:20 DevID:724
[TX] timestamp:2837.17 X:391.675 Y:-283.426 Z:1.2 SF:12 FCnt:20 DevID:831
[TX] timestamp:2837.22 X:-738.937 Y:-485.112 Z:1.2 SF:12 FCnt:19 DevID:950
[TX] timestamp:2837.42 X:746.141 Y:-277.532 Z:1.2 SF:9 FCnt:19 DevID:381
[TX] timestamp:2837.54 X:81.7345 Y:415.861 Z:1.2 SF:7 FCnt:20 DevID:213
[TX] timestamp:2837.72 X:-860.745 Y:-197.434 Z:1.2 SF:9 FCnt:19 DevID:389

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:19 DevID:159
FCnt:19 DevID:412
FCnt:19 DevID:269
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 6
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2687.2 X:463.389 Y:-236.677 Z:1.2 SF:9 FCnt:18 DevID:426
[TX] timestamp:2687.53 X:66.113 Y:-738.793 Z:1.2 SF:11 FCnt:18 DevID:728
[TX] timestamp:2687.54 X:144.319 Y:-575.014 Z:1.2 SF:11 FCnt:19 DevID:662
[TX] timestamp:2687.59 X:-973.318 Y:53.1513 Z:1.2 SF:7 FCnt:18 DevID:104
[TX] timestamp:2687.7 X:-193.57 Y:-4.39208 Z:1.2 SF:9 FCnt:19 DevID:437
[TX] timestamp:2687.71 X:-450.642 Y:582.25 Z:1.2 SF:9 FCnt:18 DevID:485
[TX] timestamp:2688.21 X:-632.002 Y:62.4065 Z:1.2 SF:9 FCnt:18 DevID:378
[TX] timestamp:2688.22 X:-640.969 Y:-217.14 Z:1.2 SF:9 FCnt:18 DevID:335
[TX] timestamp:2688.3 X:-269.892 Y:387.497 Z:1.2 SF:7 FCnt:18 DevID:132
[RX] timestamp:2688.36 SF:7 FCnt:18 DevID:132 GwID:1000 RSSI:-13.8264
[TX] timestamp:2688.55 X:494.356 Y:-38.1804 Z:1.2 SF:10 FCnt:18 DevID:554
[TX] timestamp:2688.68 X:-66.4033 Y:-308.721 Z:1.2 SF:7 FCnt:18 DevID:101
[TX] timestamp:2688.73 X:-574.57 Y:-45.9266 Z:1.2 SF:7 FCnt:18 DevID:31
[TX] timestamp:2688.82 X:207.679 Y:-570.899 Z:1.2 SF:12 FCnt:18 DevID:948
[TX] timestamp:2688.95 X:202.35 Y:-197.221 Z:1.2 SF:12 FCnt:19 DevID:879

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:18 DevID:132
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 7
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2939.8 X:-262.179 Y:-251.052 Z:1.2 SF:7 FCnt:21 DevID:261
[TX] timestamp:2939.84 X:659.341 Y:232.491 Z:1.2 SF:11 FCnt:20 DevID:720
[TX] timestamp:2939.84 X:659.341 Y:232.491 Z:1.2 SF:11 FCnt:20 DevID:720
[RX] timestamp:2939.86 SF:7 FCnt:21 DevID:261 GwID:1000 RSSI:-13.3912
[TX] timestamp:2940.13 X:-687.809 Y:527.238 Z:1.2 SF:12 FCnt:21 DevID:959
[TX] timestamp:2940.18 X:-495.256 Y:769.585 Z:1.2 SF:10 FCnt:20 DevID:549
[TX] timestamp:2940.25 X:247.847 Y:-343.017 Z:1.2 SF:7 FCnt:20 DevID:6
[TX] timestamp:2940.32 X:784.808 Y:590.025 Z:1.2 SF:10 FCnt:20 DevID:545
[TX] timestamp:2940.6 X:-365.083 Y:-172.615 Z:1.2 SF:12 FCnt:20 DevID:942
[TX] timestamp:2940.83 X:530.399 Y:281.093 Z:1.2 SF:9 FCnt:21 DevID:474
[TX] timestamp:2940.87 X:-811.7 Y:-375.762 Z:1.2 SF:9 FCnt:20 DevID:480
[TX] timestamp:2941.31 X:884.556 Y:-83.639 Z:1.2 SF:12 FCnt:20 DevID:851
[TX] timestamp:2941.4 X:10.9773 Y:-7.15517 Z:1.2 SF:9 FCnt:20 DevID:347
[TX] timestamp:2941.47 X:676.529 Y:-732.053 Z:1.2 SF:7 FCnt:20 DevID:76
[TX] timestamp:2941.62 X:131.58 Y:712.52 Z:1.2 SF:7 FCnt:20 DevID:206
[TX] timestamp:2941.73 X:294.92 Y:-404.014 Z:1.2 SF:10 FCnt:20 DevID:610
[TX] timestamp:2941.9 X:0.600433 Y:-808.719 Z:1.2 SF:7 FCnt:20 DevID:26
[RX] timestamp:2942.14 SF:10 FCnt:20 DevID:610 GwID:1000 RSSI:-16.1366

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:21 DevID:261
FCnt:20 DevID:610
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 8
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2973.04 X:380.453 Y:737.582 Z:1.2 SF:7 FCnt:20 DevID:239
[RX] timestamp:2973.11 SF:7 FCnt:20 DevID:239 GwID:1000 RSSI:-22.4215
[TX] timestamp:2973.37 X:950.006 Y:34.9957 Z:1.2 SF:7 FCnt:19 DevID:49
[RX] timestamp:2973.43 SF:7 FCnt:19 DevID:49 GwID:1000 RSSI:-27.0066
[TX] timestamp:2973.52 X:-594.06 Y:-497.065 Z:1.2 SF:12 FCnt:21 DevID:961
[TX] timestamp:2973.53 X:-561.566 Y:-78.0621 Z:1.2 SF:7 FCnt:20 DevID:65
[TX] timestamp:2973.57 X:221.211 Y:794.859 Z:1.2 SF:10 FCnt:20 DevID:618
[TX] timestamp:2973.64 X:-251.097 Y:-576.942 Z:1.2 SF:7 FCnt:20 DevID:129
[TX] timestamp:2973.72 X:-648.305 Y:-359.044 Z:1.2 SF:8 FCnt:21 DevID:306
[TX] timestamp:2973.94 X:-360.363 Y:222.454 Z:1.2 SF:7 FCnt:20 DevID:21
[TX] timestamp:2973.99 X:562.257 Y:-523.814 Z:1.2 SF:7 FCnt:21 DevID:14
[TX] timestamp:2974.02 X:841.612 Y:0.706776 Z:1.2 SF:9 FCnt:21 DevID:388
[TX] timestamp:2974.25 X:436.899 Y:337.666 Z:1.2 SF:11 FCnt:20 DevID:775
[TX] timestamp:2974.37 X:681.769 Y:448.928 Z:1.2 SF:9 FCnt:21 DevID:413

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]
FCnt:20 DevID:239
FCnt:19 DevID:49
</s>

[INST]
--------------------------------------------------------------------------
SHOT 9
--------------------------------------------------------------------------
LOG:
[TX] timestamp:3177.18 X:422.176 Y:865.413 Z:1.2 SF:8 FCnt:22 DevID:267
[TX] timestamp:3177.22 X:232.593 Y:521.208 Z:1.2 SF:11 FCnt:22 DevID:755
[RX] timestamp:3177.29 SF:8 FCnt:22 DevID:267 GwID:1000 RSSI:-24.099
[TX] timestamp:3177.59 X:-768.278 Y:-235.383 Z:1.2 SF:12 FCnt:22 DevID:911
[TX] timestamp:3177.69 X:-562.133 Y:521.936 Z:1.2 SF:10 FCnt:21 DevID:636
[TX] timestamp:3177.7 X:-193.57 Y:-4.39208 Z:1.2 SF:9 FCnt:22 DevID:437
[TX] timestamp:3177.83 X:-549.383 Y:728.22 Z:1.2 SF:12 FCnt:22 DevID:957
[TX] timestamp:3178.44 X:237.796 Y:792.457 Z:1.2 SF:11 FCnt:22 DevID:742
[TX] timestamp:3178.62 X:368.877 Y:-106.985 Z:1.2 SF:11 FCnt:22 DevID:795
[TX] timestamp:3178.68 X:-2.33003 Y:739.335 Z:1.2 SF:11 FCnt:22 DevID:770
[TX] timestamp:3178.74 X:-289.754 Y:-807.628 Z:1.2 SF:9 FCnt:21 DevID:452
[TX] timestamp:3178.88 X:-643.102 Y:221.768 Z:1.2 SF:12 FCnt:22 DevID:835
[TX] timestamp:3178.93 X:3.59957 Y:-792.762 Z:1.2 SF:11 FCnt:22 DevID:787
[TX] timestamp:3178.99 X:-32.8169 Y:-67.8536 Z:1.2 SF:9 FCnt:22 DevID:384
[RX] timestamp:3179.07 SF:12 FCnt:22 DevID:911 GwID:1000 RSSI:-22.7489
[TX] timestamp:3179.12 X:-88.401 Y:130.369 Z:1.2 SF:10 FCnt:22 DevID:533
[TX] timestamp:3179.2 X:6.13194 Y:200.329 Z:1.2 SF:9 FCnt:21 DevID:332
[TX] timestamp:3179.37 X:34.5341 Y:253.093 Z:1.2 SF:9 FCnt:22 DevID:386
[RX] timestamp:3179.53 SF:10 FCnt:22 DevID:533 GwID:1000 RSSI:-4.61995

QUESTION:
Which packets for given device are correctly transmitted?

ANSWER:[/INST]

"""

In [ ]:
response = lcpp_llm(
        prompt=prompt,
        max_tokens = None,
        #temperature= 0.9,
        #logprobs=1,
      )
answer = str(response["choices"][0]["text"]).strip().lower()
#print(response)
print(answer)


llama_print_timings:        load time =    1094.70 ms
llama_print_timings:      sample time =     180.62 ms /   249 runs   (    0.73 ms per token,  1378.55 tokens per second)
llama_print_timings: prompt eval time =   12263.89 ms /  7543 tokens (    1.63 ms per token,   615.06 tokens per second)
llama_print_timings:        eval time =   26186.06 ms /   248 runs   (  105.59 ms per token,     9.47 tokens per second)
llama_print_timings:       total time =   66953.28 ms /  7791 tokens


fcnt:22 devid:267, fcnt:22 devid:911, fcnt:22 devid:957, fcnt:22 devid:742, fcnt:22 devid:795, fcnt:22 devid:770, fcnt:21 devid:452, fcnt:22 devid:835, fcnt:22 devid:787, fcnt:22 devid:384, fcnt:21 devid:386, fcnt:22 devid:332, fcnt:21 devid:533, fcnt:22 devid:386

a total of 16 packets were correctly transmitted for the given devices with ids 267, 911, 957, 742, 795, 770, 452, 835, 787, 384, 332, and 533.


#Prompt #4
Viene cambiata la parola da "transmitted" a "received" nella domanda.

In [ ]:
prompt = """<s>
[INST]
--------------------------------------------------------------------------
SHOT 1
--------------------------------------------------------------------------
LOG:
[TX] timestamp:0.6468 X:-121.766 Y:507.761 Z:1.2 SF:7 FCnt:1 DevID:201
[RX] timestamp:0.713617 SF:7 FCnt:1 DevID:201 GwID:1000 RSSI:-18.008
[TX] timestamp:0.827825 X:781.281 Y:542.276 Z:1.2 SF:7 FCnt:1 DevID:194
[RX] timestamp:0.894644 SF:7 FCnt:1 DevID:194 GwID:1000 RSSI:-23.8409
[TX] timestamp:1.06166 X:-333.009 Y:791.847 Z:1.2 SF:8 FCnt:1 DevID:313
[RX] timestamp:1.17482 SF:8 FCnt:1 DevID:313 GwID:1000 RSSI:-25.5281
[TX] timestamp:1.38418 X:-422.336 Y:-40.1951 Z:1.2 SF:8 FCnt:1 DevID:275
[TX] timestamp:1.44612 X:-26.2489 Y:291.458 Z:1.2 SF:9 FCnt:1 DevID:324
[RX] timestamp:1.49734 SF:8 FCnt:1 DevID:275 GwID:1000 RSSI:-15.5825
[TX] timestamp:1.63083 X:-657.862 Y:-257.213 Z:1.2 SF:9 FCnt:1 DevID:459

QUESTION:
Which packets for given device are correctly received?

ANSWER:[/INST]
FCnt:1 DevID:201
FCnt:1 DevID:194
FCnt:1 DevID:313
FCnt:1 DevID:275
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 2
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2648.73 X:-440.021 Y:869.082 Z:1.2 SF:9 FCnt:18 DevID:397
[TX] timestamp:2648.81 X:311.122 Y:52.3937 Z:1.2 SF:9 FCnt:19 DevID:354
[TX] timestamp:2649.04 X:-666.532 Y:409.071 Z:1.2 SF:7 FCnt:18 DevID:127
[RX] timestamp:2649.11 SF:7 FCnt:18 DevID:127 GwID:1000 RSSI:-22.4458
[TX] timestamp:2649.28 X:-686.965 Y:424.943 Z:1.2 SF:10 FCnt:18 DevID:556
[TX] timestamp:2649.4 X:-500.989 Y:-281.463 Z:1.2 SF:12 FCnt:18 DevID:953
[TX] timestamp:2649.55 X:482.547 Y:661.83 Z:1.2 SF:9 FCnt:19 DevID:461
[TX] timestamp:2649.68 X:-823.016 Y:236.865 Z:1.2 SF:12 FCnt:19 DevID:938
[RX] timestamp:2649.69 SF:10 FCnt:18 DevID:556 GwID:1000 RSSI:-22.8529
[TX] timestamp:2649.76 X:-132.851 Y:789.532 Z:1.2 SF:7 FCnt:18 DevID:209
[TX] timestamp:2650.12 X:-105.438 Y:-818.834 Z:1.2 SF:12 FCnt:18 DevID:882
[TX] timestamp:2650.27 X:185.458 Y:-191.048 Z:1.2 SF:7 FCnt:20 DevID:173
[TX] timestamp:2650.52 X:419.132 Y:-198.629 Z:1.2 SF:8 FCnt:18 DevID:310
[TX] timestamp:2650.54 X:608.034 Y:200.211 Z:1.2 SF:12 FCnt:18 DevID:991

QUESTION:
Which packets for given device are correctly received?

ANSWER:[/INST]
FCnt:18 DevID:127
FCnt:18 DevID:556
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 3
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2653.45 X:-234.757 Y:837.144 Z:1.2 SF:12 FCnt:17 DevID:820
[TX] timestamp:2653.63 X:-376.778 Y:-284.276 Z:1.2 SF:10 FCnt:18 DevID:567
[TX] timestamp:2653.72 X:-346.365 Y:573.88 Z:1.2 SF:11 FCnt:19 DevID:694
[TX] timestamp:2654.02 X:-534.374 Y:-815.789 Z:1.2 SF:11 FCnt:19 DevID:713
[TX] timestamp:2654.11 X:621.242 Y:595.208 Z:1.2 SF:12 FCnt:18 DevID:833
[TX] timestamp:2654.34 X:112.828 Y:-538.224 Z:1.2 SF:10 FCnt:19 DevID:501
[TX] timestamp:2654.46 X:592.428 Y:239.434 Z:1.2 SF:11 FCnt:19 DevID:654
[TX] timestamp:2654.74 X:-587.472 Y:533.362 Z:1.2 SF:12 FCnt:18 DevID:949
[TX] timestamp:2654.8 X:252.053 Y:852.557 Z:1.2 SF:7 FCnt:18 DevID:234
[TX] timestamp:2655.14 X:-749.124 Y:-461.172 Z:1.2 SF:7 FCnt:18 DevID:227
[RX] timestamp:2655.21 SF:7 FCnt:18 DevID:227 GwID:1000 RSSI:-28.7341
[TX] timestamp:2655.23 X:371.431 Y:-279.928 Z:1.2 SF:7 FCnt:18 DevID:81
[RX] timestamp:2655.29 SF:7 FCnt:18 DevID:81 GwID:1000 RSSI:-18.1152
[TX] timestamp:2655.56 X:-398.763 Y:712.809 Z:1.2 SF:12 FCnt:19 DevID:976
[TX] timestamp:2655.57 X:230.417 Y:-424.954 Z:1.2 SF:9 FCnt:18 DevID:365
[TX] timestamp:2655.62 X:131.58 Y:712.52 Z:1.2 SF:7 FCnt:18 DevID:206

QUESTION:
Which packets for given device are correctly received?

ANSWER:[/INST]
FCnt:18 DevID:227
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 4
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2687.2 X:463.389 Y:-236.677 Z:1.2 SF:9 FCnt:18 DevID:426
[TX] timestamp:2687.53 X:66.113 Y:-738.793 Z:1.2 SF:11 FCnt:18 DevID:728
[TX] timestamp:2687.54 X:144.319 Y:-575.014 Z:1.2 SF:11 FCnt:19 DevID:662
[TX] timestamp:2687.59 X:-973.318 Y:53.1513 Z:1.2 SF:7 FCnt:18 DevID:104
[TX] timestamp:2687.7 X:-193.57 Y:-4.39208 Z:1.2 SF:9 FCnt:19 DevID:437
[TX] timestamp:2687.71 X:-450.642 Y:582.25 Z:1.2 SF:9 FCnt:18 DevID:485
[TX] timestamp:2688.21 X:-632.002 Y:62.4065 Z:1.2 SF:9 FCnt:18 DevID:378
[TX] timestamp:2688.22 X:-640.969 Y:-217.14 Z:1.2 SF:9 FCnt:18 DevID:335
[TX] timestamp:2688.3 X:-269.892 Y:387.497 Z:1.2 SF:7 FCnt:18 DevID:132
[RX] timestamp:2688.36 SF:7 FCnt:18 DevID:132 GwID:1000 RSSI:-13.8264
[TX] timestamp:2688.55 X:494.356 Y:-38.1804 Z:1.2 SF:10 FCnt:18 DevID:554
[TX] timestamp:2688.68 X:-66.4033 Y:-308.721 Z:1.2 SF:7 FCnt:18 DevID:101
[TX] timestamp:2688.73 X:-574.57 Y:-45.9266 Z:1.2 SF:7 FCnt:18 DevID:31
[TX] timestamp:2688.82 X:207.679 Y:-570.899 Z:1.2 SF:12 FCnt:18 DevID:948
[TX] timestamp:2688.95 X:202.35 Y:-197.221 Z:1.2 SF:12 FCnt:19 DevID:879

QUESTION:
Which packets for given device are correctly received?

ANSWER:[/INST]
FCnt:18 DevID:132
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 5
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2836.12 X:-763.089 Y:303.751 Z:1.2 SF:9 FCnt:19 DevID:340
[RX] timestamp:2836.34 SF:9 FCnt:19 DevID:340 GwID:1000 RSSI:-22.1819
[TX] timestamp:2836.36 X:667.276 Y:209.04 Z:1.2 SF:7 FCnt:19 DevID:159
[TX] timestamp:2836.39 X:-36.1486 Y:531.612 Z:1.2 SF:11 FCnt:19 DevID:648
[TX] timestamp:2836.4 X:533.818 Y:710.866 Z:1.2 SF:9 FCnt:19 DevID:338
[RX] timestamp:2836.42 SF:7 FCnt:19 DevID:159 GwID:1000 RSSI:-24.4854
[TX] timestamp:2836.5 X:602.687 Y:-299.853 Z:1.2 SF:9 FCnt:19 DevID:412
[TX] timestamp:2836.64 X:-497.759 Y:424.351 Z:1.2 SF:11 FCnt:20 DevID:682
[RX] timestamp:2836.72 SF:9 FCnt:19 DevID:412 GwID:1000 RSSI:-23.1671
[TX] timestamp:2836.8 X:647.59 Y:-302.591 Z:1.2 SF:8 FCnt:19 DevID:269
[TX] timestamp:2836.84 X:-932.651 Y:-299.507 Z:1.2 SF:10 FCnt:19 DevID:500
[RX] timestamp:2836.91 SF:8 FCnt:19 DevID:269 GwID:1000 RSSI:-20.7882
[TX] timestamp:2837.09 X:463.067 Y:122.925 Z:1.2 SF:11 FCnt:20 DevID:724
[TX] timestamp:2837.17 X:391.675 Y:-283.426 Z:1.2 SF:12 FCnt:20 DevID:831
[TX] timestamp:2837.22 X:-738.937 Y:-485.112 Z:1.2 SF:12 FCnt:19 DevID:950
[TX] timestamp:2837.42 X:746.141 Y:-277.532 Z:1.2 SF:9 FCnt:19 DevID:381
[TX] timestamp:2837.54 X:81.7345 Y:415.861 Z:1.2 SF:7 FCnt:20 DevID:213
[TX] timestamp:2837.72 X:-860.745 Y:-197.434 Z:1.2 SF:9 FCnt:19 DevID:389

QUESTION:
Which packets for given device are correctly received?

ANSWER:[/INST]
FCnt:19 DevID:159
FCnt:19 DevID:412
FCnt:19 DevID:269
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 6
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2687.2 X:463.389 Y:-236.677 Z:1.2 SF:9 FCnt:18 DevID:426
[TX] timestamp:2687.53 X:66.113 Y:-738.793 Z:1.2 SF:11 FCnt:18 DevID:728
[TX] timestamp:2687.54 X:144.319 Y:-575.014 Z:1.2 SF:11 FCnt:19 DevID:662
[TX] timestamp:2687.59 X:-973.318 Y:53.1513 Z:1.2 SF:7 FCnt:18 DevID:104
[TX] timestamp:2687.7 X:-193.57 Y:-4.39208 Z:1.2 SF:9 FCnt:19 DevID:437
[TX] timestamp:2687.71 X:-450.642 Y:582.25 Z:1.2 SF:9 FCnt:18 DevID:485
[TX] timestamp:2688.21 X:-632.002 Y:62.4065 Z:1.2 SF:9 FCnt:18 DevID:378
[TX] timestamp:2688.22 X:-640.969 Y:-217.14 Z:1.2 SF:9 FCnt:18 DevID:335
[TX] timestamp:2688.3 X:-269.892 Y:387.497 Z:1.2 SF:7 FCnt:18 DevID:132
[RX] timestamp:2688.36 SF:7 FCnt:18 DevID:132 GwID:1000 RSSI:-13.8264
[TX] timestamp:2688.55 X:494.356 Y:-38.1804 Z:1.2 SF:10 FCnt:18 DevID:554
[TX] timestamp:2688.68 X:-66.4033 Y:-308.721 Z:1.2 SF:7 FCnt:18 DevID:101
[TX] timestamp:2688.73 X:-574.57 Y:-45.9266 Z:1.2 SF:7 FCnt:18 DevID:31
[TX] timestamp:2688.82 X:207.679 Y:-570.899 Z:1.2 SF:12 FCnt:18 DevID:948
[TX] timestamp:2688.95 X:202.35 Y:-197.221 Z:1.2 SF:12 FCnt:19 DevID:879

QUESTION:
Which packets for given device are correctly received?

ANSWER:[/INST]
FCnt:18 DevID:132
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 7
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2939.8 X:-262.179 Y:-251.052 Z:1.2 SF:7 FCnt:21 DevID:261
[TX] timestamp:2939.84 X:659.341 Y:232.491 Z:1.2 SF:11 FCnt:20 DevID:720
[TX] timestamp:2939.84 X:659.341 Y:232.491 Z:1.2 SF:11 FCnt:20 DevID:720
[RX] timestamp:2939.86 SF:7 FCnt:21 DevID:261 GwID:1000 RSSI:-13.3912
[TX] timestamp:2940.13 X:-687.809 Y:527.238 Z:1.2 SF:12 FCnt:21 DevID:959
[TX] timestamp:2940.18 X:-495.256 Y:769.585 Z:1.2 SF:10 FCnt:20 DevID:549
[TX] timestamp:2940.25 X:247.847 Y:-343.017 Z:1.2 SF:7 FCnt:20 DevID:6
[TX] timestamp:2940.32 X:784.808 Y:590.025 Z:1.2 SF:10 FCnt:20 DevID:545
[TX] timestamp:2940.6 X:-365.083 Y:-172.615 Z:1.2 SF:12 FCnt:20 DevID:942
[TX] timestamp:2940.83 X:530.399 Y:281.093 Z:1.2 SF:9 FCnt:21 DevID:474
[TX] timestamp:2940.87 X:-811.7 Y:-375.762 Z:1.2 SF:9 FCnt:20 DevID:480
[TX] timestamp:2941.31 X:884.556 Y:-83.639 Z:1.2 SF:12 FCnt:20 DevID:851
[TX] timestamp:2941.4 X:10.9773 Y:-7.15517 Z:1.2 SF:9 FCnt:20 DevID:347
[TX] timestamp:2941.47 X:676.529 Y:-732.053 Z:1.2 SF:7 FCnt:20 DevID:76
[TX] timestamp:2941.62 X:131.58 Y:712.52 Z:1.2 SF:7 FCnt:20 DevID:206
[TX] timestamp:2941.73 X:294.92 Y:-404.014 Z:1.2 SF:10 FCnt:20 DevID:610
[TX] timestamp:2941.9 X:0.600433 Y:-808.719 Z:1.2 SF:7 FCnt:20 DevID:26
[RX] timestamp:2942.14 SF:10 FCnt:20 DevID:610 GwID:1000 RSSI:-16.1366

QUESTION:
Which packets for given device are correctly received?

ANSWER:[/INST]
FCnt:21 DevID:261
FCnt:20 DevID:610
</s>

<s>
[INST]
--------------------------------------------------------------------------
SHOT 8
--------------------------------------------------------------------------
LOG:
[TX] timestamp:2973.04 X:380.453 Y:737.582 Z:1.2 SF:7 FCnt:20 DevID:239
[RX] timestamp:2973.11 SF:7 FCnt:20 DevID:239 GwID:1000 RSSI:-22.4215
[TX] timestamp:2973.37 X:950.006 Y:34.9957 Z:1.2 SF:7 FCnt:19 DevID:49
[RX] timestamp:2973.43 SF:7 FCnt:19 DevID:49 GwID:1000 RSSI:-27.0066
[TX] timestamp:2973.52 X:-594.06 Y:-497.065 Z:1.2 SF:12 FCnt:21 DevID:961
[TX] timestamp:2973.53 X:-561.566 Y:-78.0621 Z:1.2 SF:7 FCnt:20 DevID:65
[TX] timestamp:2973.57 X:221.211 Y:794.859 Z:1.2 SF:10 FCnt:20 DevID:618
[TX] timestamp:2973.64 X:-251.097 Y:-576.942 Z:1.2 SF:7 FCnt:20 DevID:129
[TX] timestamp:2973.72 X:-648.305 Y:-359.044 Z:1.2 SF:8 FCnt:21 DevID:306
[TX] timestamp:2973.94 X:-360.363 Y:222.454 Z:1.2 SF:7 FCnt:20 DevID:21
[TX] timestamp:2973.99 X:562.257 Y:-523.814 Z:1.2 SF:7 FCnt:21 DevID:14
[TX] timestamp:2974.02 X:841.612 Y:0.706776 Z:1.2 SF:9 FCnt:21 DevID:388
[TX] timestamp:2974.25 X:436.899 Y:337.666 Z:1.2 SF:11 FCnt:20 DevID:775
[TX] timestamp:2974.37 X:681.769 Y:448.928 Z:1.2 SF:9 FCnt:21 DevID:413

QUESTION:
Which packets for given device are correctly received?

ANSWER:[/INST]
FCnt:20 DevID:239
FCnt:19 DevID:49
</s>

[INST]
--------------------------------------------------------------------------
SHOT 9
--------------------------------------------------------------------------
LOG:
[TX] timestamp:3177.18 X:422.176 Y:865.413 Z:1.2 SF:8 FCnt:22 DevID:267
[TX] timestamp:3177.22 X:232.593 Y:521.208 Z:1.2 SF:11 FCnt:22 DevID:755
[RX] timestamp:3177.29 SF:8 FCnt:22 DevID:267 GwID:1000 RSSI:-24.099
[TX] timestamp:3177.59 X:-768.278 Y:-235.383 Z:1.2 SF:12 FCnt:22 DevID:911
[TX] timestamp:3177.69 X:-562.133 Y:521.936 Z:1.2 SF:10 FCnt:21 DevID:636
[TX] timestamp:3177.7 X:-193.57 Y:-4.39208 Z:1.2 SF:9 FCnt:22 DevID:437
[TX] timestamp:3177.83 X:-549.383 Y:728.22 Z:1.2 SF:12 FCnt:22 DevID:957
[TX] timestamp:3178.44 X:237.796 Y:792.457 Z:1.2 SF:11 FCnt:22 DevID:742
[TX] timestamp:3178.62 X:368.877 Y:-106.985 Z:1.2 SF:11 FCnt:22 DevID:795
[TX] timestamp:3178.68 X:-2.33003 Y:739.335 Z:1.2 SF:11 FCnt:22 DevID:770
[TX] timestamp:3178.74 X:-289.754 Y:-807.628 Z:1.2 SF:9 FCnt:21 DevID:452
[TX] timestamp:3178.88 X:-643.102 Y:221.768 Z:1.2 SF:12 FCnt:22 DevID:835
[TX] timestamp:3178.93 X:3.59957 Y:-792.762 Z:1.2 SF:11 FCnt:22 DevID:787
[TX] timestamp:3178.99 X:-32.8169 Y:-67.8536 Z:1.2 SF:9 FCnt:22 DevID:384
[RX] timestamp:3179.07 SF:12 FCnt:22 DevID:911 GwID:1000 RSSI:-22.7489
[TX] timestamp:3179.12 X:-88.401 Y:130.369 Z:1.2 SF:10 FCnt:22 DevID:533
[TX] timestamp:3179.2 X:6.13194 Y:200.329 Z:1.2 SF:9 FCnt:21 DevID:332
[TX] timestamp:3179.37 X:34.5341 Y:253.093 Z:1.2 SF:9 FCnt:22 DevID:386
[RX] timestamp:3179.53 SF:10 FCnt:22 DevID:533 GwID:1000 RSSI:-4.61995

QUESTION:
Which packets for given device are correctly received?

ANSWER:[/INST]

"""

In [ ]:
from transformers import AutoTokenizer, MistralForCausalLM

model = MistralForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
response = lcpp_llm(
        prompt=prompt,
        max_tokens = None,
        #temperature= 0.9,
        #logprobs=1,
      )
answer = str(response["choices"][0]["text"]).strip().lower()
#print(response)
print(answer)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1094.70 ms
llama_print_timings:      sample time =      53.84 ms /    87 runs   (    0.62 ms per token,  1615.93 tokens per second)
llama_print_timings: prompt eval time =   12092.34 ms /  7019 tokens (    1.72 ms per token,   580.45 tokens per second)
llama_print_timings:        eval time =    8857.12 ms /    86 runs   (  102.99 ms per token,     9.71 tokens per second)
llama_print_timings:       total time =   45501.71 ms /  7105 tokens


fcnt:22 devid:267 (received at timestamp 3177.29)
fcnt:22 devid:911 (received at timestamp 3177.59 and 3179.07)
fcnt:22 devid:533 (received at timestamp 3179.53)
